# 导入数据

In [48]:
# 导入相关package
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import chardet

In [49]:
# 定义一个函数，自动检测文件编码并读取文件
def read_csv_with_detected_encoding(file_path):
    # 检测文件编码
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
        encoding = result['encoding']
        print(f"检测到文件 {file_path} 的编码格式为：{encoding}")
    # 使用检测到的编码读取文件
    return pd.read_csv(file_path, encoding=encoding)

# 读取 data_dictionary.csv 文件
csv_content = read_csv_with_detected_encoding('2025_Problem_C_Data\\data_dictionary.csv')
print("data_dictionary.csv 数据预览：")
print(csv_content.head())

# 读取 summerOly_medal_counts.csv 文件
medal_counts = read_csv_with_detected_encoding('2025_Problem_C_Data\\summerOly_medal_counts.csv')
print("\nsummerOly_medal_counts.csv 数据预览：")
print(medal_counts.head())

# 读取 summerOly_hosts.csv 文件
olympic_hosts = read_csv_with_detected_encoding('2025_Problem_C_Data\\summerOly_hosts.csv')
print("\nsummerOly_hosts.csv 数据预览：")
print(olympic_hosts.head())

# 读取 summerOly_programs.csv 文件
olympic_programs = read_csv_with_detected_encoding('2025_Problem_C_Data\\summerOly_programs.csv')
print("\nsummerOly_programs.csv 数据预览：")
print(olympic_programs.head())

# 读取 summerOly_athletes.csv 文件
olympic_athletes = read_csv_with_detected_encoding('2025_Problem_C_Data\\summerOly_athletes.csv')
print("\nsummerOly_athletes.csv 数据预览：")
print(olympic_athletes.head())

检测到文件 2025_Problem_C_Data\data_dictionary.csv 的编码格式为：Windows-1252
data_dictionary.csv 数据预览：
  summerOly_medal_counts.csv                                     Unnamed: 1  \
0                  variables                                    explanation   
1                       Rank      Rank of country based on total medals won   
2                        NOC  Name of country as recorded for that Olympics   
3                       Gold       Number of Gold medals the country earned   
4                     Silver     Number of Silver medals the country earned   

      Unnamed: 2  
0        example  
1           1, 2  
2  China, France  
3        0, 1, 2  
4        0, 1, 2  
检测到文件 2025_Problem_C_Data\summerOly_medal_counts.csv 的编码格式为：utf-8

summerOly_medal_counts.csv 数据预览：
   Rank            NOC  Gold  Silver  Bronze  Total  Year
0     1  United States    11       7       2     20  1896
1     2         Greece    10      18      19     47  1896
2     3        Germany     6       5       2 

# 数据清洗

### 缺失值检查

In [50]:
# 1. 缺失值检查
def check_missing_values(file_path):
    """
    检查 CSV 文件中的缺失值。

    参数:
        file_path (str): CSV 文件的路径。

    返回:
        None，但会打印缺失值的相关信息。
    """
    try:
        # 尝试读取 CSV 文件
        data = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        data = pd.read_csv(file_path, encoding='ISO-8859-1')

    print(file_path)

    # 检查每列的缺失值数量
    missing_values_per_column = data.isnull().sum()
    print("每列的缺失值数量：")
    print(missing_values_per_column)

    # 检查整个数据框的总缺失值数量
    total_missing_values = missing_values_per_column.sum()
    print("整个数据框的总缺失值数量：", total_missing_values)

    # 检查是否有任何缺失值
    has_missing_values = data.isnull().values.any()
    print("数据框中是否存在缺失值：", has_missing_values)
    print("\n")

    # 如果有缺失值，输出包含缺失值的行
    if has_missing_values:
        print("\n包含缺失值的行：")
        print(data[data.isnull().any(axis=1)])
    
content_name = ['2025_Problem_C_Data\\summerOly_medal_counts.csv', '2025_Problem_C_Data\\summerOly_hosts.csv', '2025_Problem_C_Data\\summerOly_programs.csv', '2025_Problem_C_Data\\summerOly_athletes.csv']
for i in content_name:
    check_missing_values(i)

2025_Problem_C_Data\summerOly_medal_counts.csv
每列的缺失值数量：
Rank      0
NOC       0
Gold      0
Silver    0
Bronze    0
Total     0
Year      0
dtype: int64
整个数据框的总缺失值数量： 0
数据框中是否存在缺失值： False


2025_Problem_C_Data\summerOly_hosts.csv
每列的缺失值数量：
Year    0
Host    0
dtype: int64
整个数据框的总缺失值数量： 0
数据框中是否存在缺失值： False


2025_Problem_C_Data\summerOly_programs.csv
每列的缺失值数量：
Sport                    0
Discipline               2
Code                     0
Sports Governing Body    0
1896                     0
1900                     0
1904                     0
1906*                    0
1908                     0
1912                     0
1920                     0
1924                     0
1928                     2
1932                     2
1936                     2
1948                     2
1952                     2
1956                     2
1960                     2
1964                     2
1968                     2
1972                     2
1976                     2
1980           

### 补全summerOly_programs.csv中的缺失值

In [51]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import re
import os

# 确保保存结果的目录存在
os.makedirs('Generated', exist_ok=True)

data = olympic_programs.copy()

# 3. 检查缺失值
#print("每列的缺失值数量：")
#print(data.isnull().sum())

# 4. 填充 Discipline 列的缺失值
data['Discipline'] = data['Discipline'].fillna(data['Sport'])

# 5. 准备年份列的数据
years = [col for col in data.columns if col.isdigit() or col.endswith('*')]

# 6. 将数据从宽格式转换为长格式
data_long = data.melt(id_vars=['Sport', 'Discipline', 'Code', 'Sports Governing Body'], 
                      value_vars=years, 
                      var_name='Year', 
                      value_name='Events')

# 7. 将年份列转换为数值
data_long['Year'] = data_long['Year'].str.replace('*', '').astype(int)

# 8. 清理 Events 列中的非数值字符
def clean_events(value):
    if isinstance(value, str):
        # 移除非数值字符
        cleaned_value = re.sub(r'[^0-9]', '', value)
        return float(cleaned_value) if cleaned_value.isdigit() else np.nan
    return value

data_long['Events'] = data_long['Events'].apply(clean_events)

# 9. 将1924年以及之后的 Skating 和 Ice Hockey 项目的赛事数目填为0
mask = (data_long['Year'] >= 1924) & (data_long['Sport'].isin(['Skating', 'Ice Hockey']))
data_long.loc[mask, 'Events'] = 0

# 10. 分组处理，按运动种类单独训练模型
for sport, group in data_long.groupby('Sport'):
    # 分离已知数据和缺失数据
    known_data = group.dropna(subset=['Events'])
    missing_data = group[group['Events'].isna()]
    
    if not known_data.empty and not missing_data.empty:
        # 准备训练数据
        X_known = known_data[['Year']]
        y_known = known_data['Events']
        
        # 检查已知数据的数量
        if len(y_known) < 5:
            print(f"警告：运动种类 '{sport}' 的已知数据太少，使用 KNN 或线性回归填充。")
            
            # 尝试使用线性回归
            if len(y_known) >= 3:  # 至少需要3个点来拟合线性回归
                model = LinearRegression()
                model.fit(X_known, y_known)
                predicted_events = model.predict(missing_data[['Year']])
            else:  # 使用 KNN，K=1
                model = KNeighborsRegressor(n_neighbors=1)
                model.fit(X_known, y_known)
                predicted_events = model.predict(missing_data[['Year']])
            
            # 将预测值四舍五入为整数
            predicted_events = np.round(predicted_events).astype(int)
            
            # 将预测值转换为 Pandas Series，并确保索引对齐
            predicted_series = pd.Series(predicted_events, index=missing_data.index)
            
            # 填充缺失值
            data_long.loc[data_long['Sport'] == sport, 'Events'] = data_long.loc[data_long['Sport'] == sport, 'Events'].fillna(predicted_series)
        else:
            # 训练随机森林模型
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_known, y_known)
            
            # 预测缺失数据
            X_missing = missing_data[['Year']]
            predicted_events = model.predict(X_missing)
            
            # 将预测值四舍五入为整数
            predicted_events = np.round(predicted_events).astype(int)
            
            # 将预测值转换为 Pandas Series，并确保索引对齐
            predicted_series = pd.Series(predicted_events, index=missing_data.index)
            
            # 填充缺失值
            data_long.loc[data_long['Sport'] == sport, 'Events'] = data_long.loc[data_long['Sport'] == sport, 'Events'].fillna(predicted_series)
            
            # 记录日志
            print(f"运动种类 '{sport}' 的模型训练完成，预测了 {len(predicted_events)} 个缺失值。")
    else:
        print(f"运动种类 '{sport}' 没有缺失数据或没有足够的已知数据。")

# 11. 将数据重新转换为宽格式
data_filled = data_long.pivot_table(index=['Sport', 'Discipline', 'Code', 'Sports Governing Body'], 
                                    columns='Year', 
                                    values='Events', 
                                    aggfunc='first').reset_index()

# 12. 输出结果
print("\n填充后的数据：")
print(data_filled.head())

# 13. 保存结果到新的 CSV 文件
output_path = 'Generated\\summerOly_programs_filled.csv'
data_filled.to_csv(output_path, index=False, encoding='utf-8')  # 确保保存时使用正确的编码
print(f"填充后的数据已保存到 {output_path}")

运动种类 'Aquatics' 没有缺失数据或没有足够的已知数据。
运动种类 'Archery' 没有缺失数据或没有足够的已知数据。
运动种类 'Athletics' 没有缺失数据或没有足够的已知数据。
运动种类 'Badminton' 的模型训练完成，预测了 2 个缺失值。
运动种类 'Baseball and Softball' 的模型训练完成，预测了 8 个缺失值。
运动种类 'Basketball' 的模型训练完成，预测了 2 个缺失值。
运动种类 'Basque Pelota' 的模型训练完成，预测了 4 个缺失值。
运动种类 'Boxing' 没有缺失数据或没有足够的已知数据。
运动种类 'Breaking' 没有缺失数据或没有足够的已知数据。
运动种类 'Canoeing' 的模型训练完成，预测了 1 个缺失值。
运动种类 'Cricket' 没有缺失数据或没有足够的已知数据。
运动种类 'Croquet' 没有缺失数据或没有足够的已知数据。
运动种类 'Cycling' 没有缺失数据或没有足够的已知数据。
运动种类 'Equestrian' 没有缺失数据或没有足够的已知数据。
运动种类 'Fencing' 没有缺失数据或没有足够的已知数据。
运动种类 'Field hockey' 没有缺失数据或没有足够的已知数据。
运动种类 'Flag football' 没有缺失数据或没有足够的已知数据。
运动种类 'Football' 没有缺失数据或没有足够的已知数据。
运动种类 'Golf' 没有缺失数据或没有足够的已知数据。
运动种类 'Gymnastics' 没有缺失数据或没有足够的已知数据。
运动种类 'Handball' 的模型训练完成，预测了 1 个缺失值。
运动种类 'Ice Hockey' 没有缺失数据或没有足够的已知数据。
运动种类 'Jeu de Paume' 没有缺失数据或没有足够的已知数据。
运动种类 'Judo' 没有缺失数据或没有足够的已知数据。
运动种类 'Karate' 没有缺失数据或没有足够的已知数据。
运动种类 'Lacrosse' 的模型训练完成，预测了 3 个缺失值。
运动种类 'Modern Pentathlon' 没有缺失数据或没有足够的已知数据。
运动种类 'Polo' 没有缺失数据或没有足够的已知数据。
运动种类 

### Medal_counts 数据清洗

In [52]:
# 2. 数据清洗
# 确保数据的格式正确
data = medal_counts[['Year', 'NOC', 'Gold', 'Silver', 'Bronze', 'Total']]

# 3. 创建年份和国家的索引
years = data['Year'].unique()
noc = data['NOC'].unique()

# 4. 定义一个函数来生成表格
def generate_table(data, column_name):
    # 创建一个空的 DataFrame，以年份为列，国家为行
    table = pd.DataFrame(index=noc, columns=years)
    
    # 填充数据
    for index, row in data.iterrows():
        year = row['Year']
        country = row['NOC']
        value = row[column_name]
        table.at[country, year] = value
    
    # 推断数据类型并填充缺失值为0
    table = table.infer_objects(copy=False).fillna(0).astype(int)
    
    return table

# 5. 生成金牌、银牌、铜牌和总数的表格
gold_table = generate_table(data, 'Gold')
silver_table = generate_table(data, 'Silver')
bronze_table = generate_table(data, 'Bronze')
total_table = generate_table(data, 'Total')

# 6. 保存到新的 CSV 文件
gold_table.to_csv('Generated\\summerOly_gold_summary.csv')
silver_table.to_csv('Generated\\summerOly_silver_summary.csv')
bronze_table.to_csv('Generated\\summerOly_bronze_summary.csv')
total_table.to_csv('Generated\\summerOly_total_summary.csv')

In [53]:
# 7. 输出结果
print("金牌表格：")
print(gold_table)

金牌表格：
                      1896  1900  1904  1908  1912  1920  1924  1928  1932  \
United States           11    19    76    23    26    41    45    22     0   
Greece                  10     0     1     0     1     0     0     0     0   
Germany                  6     4     4     3     5     0     0    10     0   
France                   5    27     0     5     7     9    13     6     0   
Great Britain            2    15     1    56    10    14     9     3     0   
...                    ...   ...   ...   ...   ...   ...   ...   ...   ...   
Saint Lucia              0     0     0     0     0     0     0     0     0   
Dominica                 0     0     0     0     0     0     0     0     0   
Albania                  0     0     0     0     0     0     0     0     0   
Cabo Verde               0     0     0     0     0     0     0     0     0   
Refugee Olympic Team     0     0     0     0     0     0     0     0     0   

                      1936  ...  1988  1992  1996  2000  

In [54]:
print("\n银牌表格：")
print(silver_table)


银牌表格：
                      1896  1900  1904  1908  1912  1920  1924  1928  1932  \
United States            7    14    78    12    19    27    27    18     0   
Greece                  18     0     0     3     0     1     0     0     0   
Germany                  5     3     5     5    13     0     0     7     0   
France                   4    39     1     5     4    19    15    10     0   
Great Britain            3     7     1    51    15    15    13    10     0   
...                    ...   ...   ...   ...   ...   ...   ...   ...   ...   
Saint Lucia              0     0     0     0     0     0     0     0     0   
Dominica                 0     0     0     0     0     0     0     0     0   
Albania                  0     0     0     0     0     0     0     0     0   
Cabo Verde               0     0     0     0     0     0     0     0     0   
Refugee Olympic Team     0     0     0     0     0     0     0     0     0   

                      1936  ...  1988  1992  1996  2000 

In [55]:
print("\n铜牌表格：")
print(bronze_table)


铜牌表格：
                      1896  1900  1904  1908  1912  1920  1924  1928  1932  \
United States            2    15    77    12    19    27    27    16     0   
Greece                  19     0     1     1     1     0     0     0     0   
Germany                  2     2     6     5     7     0     0    14     0   
France                   2    37     0     9     3    13    10     5     0   
Great Britain            2     9     0    39    16    13    12     7     0   
...                    ...   ...   ...   ...   ...   ...   ...   ...   ...   
Saint Lucia              0     0     0     0     0     0     0     0     0   
Dominica                 0     0     0     0     0     0     0     0     0   
Albania                  0     0     0     0     0     0     0     0     0   
Cabo Verde               0     0     0     0     0     0     0     0     0   
Refugee Olympic Team     0     0     0     0     0     0     0     0     0   

                      1936  ...  1988  1992  1996  2000 

In [56]:

print("\n总数表格：")
print(total_table)


总数表格：
                      1896  1900  1904  1908  1912  1920  1924  1928  1932  \
United States           20    48   231    47    64    95    99    56     0   
Greece                  47     0     2     4     2     1     0     0     0   
Germany                 13     9    15    13    25     0     0    31     0   
France                  11   103     1    19    14    41    38    21     0   
Great Britain            7    31     2   146    41    42    34    20     0   
...                    ...   ...   ...   ...   ...   ...   ...   ...   ...   
Saint Lucia              0     0     0     0     0     0     0     0     0   
Dominica                 0     0     0     0     0     0     0     0     0   
Albania                  0     0     0     0     0     0     0     0     0   
Cabo Verde               0     0     0     0     0     0     0     0     0   
Refugee Olympic Team     0     0     0     0     0     0     0     0     0   

                      1936  ...  1988  1992  1996  2000 

### 清理athletes.csv并转换格式为宽

In [57]:
# 读取 summerOly_athletes.csv 文件
data = olympic_athletes.copy()

# 转换为长格式，将年份放到列的抬头位置
pivot_df = data.pivot_table(index=['Name', 'Sex', 'Team', 'NOC', 'City', 'Sport', 'Event'], 
                                  columns='Year', 
                                  values='Medal',
                                  aggfunc='first').reset_index()

# 填充缺失值为0
pivot_df = pivot_df.fillna(0)

# 输出结果
print("转换为宽格式后的数据：")
print(pivot_df.head())

# 保存为新的 CSV 文件
output_path = 'Generated\\summerOly_athletes_wide_format.csv'
pivot_df.to_csv(output_path, index=False, encoding='utf-8')
print(f"宽格式数据已保存到 {output_path}")

转换为宽格式后的数据：
Year           Name Sex          Team  NOC   City          Sport  \
0      (jr) Larocca   M     Argentina  ARG  Paris     Equestrian   
1     . Chadalavada   F         India  IND  Tokyo        Fencing   
2            . Deni   M     Indonesia  INA  Tokyo  Weightlifting   
3               671   F         China  CHN  Paris       Breaking   
4          A Alayed   F  Saudi Arabia  KSA  Paris       Swimming   

Year                     Event 1896 1900 1904  ... 1988 1992 1996 2000 2004  \
0           Jumping Individual    0    0    0  ...    0    0    0    0    0   
1     Women's Sabre Individual    0    0    0  ...    0    0    0    0    0   
2                   Men's 67kg    0    0    0  ...    0    0    0    0    0   
3                      B-Girls    0    0    0  ...    0    0    0    0    0   
4       Women's 200m Freestyle    0    0    0  ...    0    0    0    0    0   

Year 2008 2012 2016      2020      2024  
0       0    0    0         0  No medal  
1       0    0    0 

# 分析数据

### 统计连续参加奥运会的年数与对应人数

In [58]:
# 读取 summerOly_athletes.csv 文件
data = olympic_athletes.copy()

# 提取必要的列
athlete_years = olympic_athletes[['Name', 'Sex', 'NOC', 'Year']].drop_duplicates()

# 对每个运动员的参赛年份进行排序
athlete_years = athlete_years.sort_values(by=['Name', 'Sex', 'NOC', 'Year'])

# 计算连续参加的届数
def count_consecutive_years(group):
    years = group['Year'].values
    consecutive_years = []
    current_count = 1
    for i in range(1, len(years)):
        if years[i] - years[i - 1] <= 5:
            current_count += 1
        else:
            consecutive_years.append(current_count)
            current_count = 1
    consecutive_years.append(current_count)
    return pd.Series(consecutive_years)

# 应用函数计算每个运动员的连续届数
consecutive_years = athlete_years.groupby('Name').apply(count_consecutive_years, include_groups=False).explode().reset_index()
consecutive_years.columns = ['level_0', 'Name', 'Consecutive_Years']  # 修正列名
consecutive_years = consecutive_years.drop(columns=['level_0'])  # 删除不必要的列

# 统计每个连续届数的人数
consecutive_years_count = consecutive_years['Consecutive_Years'].value_counts().reset_index()
consecutive_years_count.columns = ['Consecutive_Years', 'Count']

# 输出结果
print("连续参加奥运会的届数与对应人次：")
print(consecutive_years_count)

# 保存为新的 CSV 文件
output_path = 'Generated\\consecutive_years_count.csv'
consecutive_years_count.to_csv(output_path, index=False, encoding='utf-8')
print(f"统计结果已保存到 {output_path}")

连续参加奥运会的届数与对应人次：
    Consecutive_Years   Count
0                   1  103847
1                   2   23835
2                   3    6483
3                   4    1764
4                   5     432
5                   6     100
6                   7      34
7                   8      10
8                   9       4
9                  10       1
10                 11       1
11                 13       1
统计结果已保存到 Generated\consecutive_years_count.csv


In [59]:
# 计算每个运动员的第一次和最后一次参赛年份
def calculate_year_gap(group):
    years = group['Year'].sort_values().values
    if len(years) > 1:
        return years[-1] - years[0] + 1
    else:
        return 1  # 如果只参加了一次，间隔为1

# 应用函数计算每个运动员的间隔年数
athlete_gaps = athlete_years.groupby('Name').apply(calculate_year_gap, include_groups=False).reset_index()
athlete_gaps.columns = ['Name', 'Year_Gap']

# 统计每个间隔年数的人数
gap_counts = athlete_gaps['Year_Gap'].value_counts().reset_index()
gap_counts.columns = ['Year_Gap', 'Count']

# 按 Year_Gap 排序
gap_counts = gap_counts.sort_values(by='Year_Gap')

# 输出结果
print("运动员第一次参加奥运会和最后一次参加奥运会之间的间隔年数：")
print(gap_counts)

# 保存为新的 CSV 文件
output_path = 'Generated\\athlete_year_gaps.csv'
gap_counts.to_csv(output_path, index=False, encoding='utf-8')
print(f"统计结果已保存到 {output_path}")

运动员第一次参加奥运会和最后一次参加奥运会之间的间隔年数：
    Year_Gap  Count
0          1  94044
11         3     97
1          5  21426
15         7     69
2          9   8160
30        11     11
3         13   3095
28        15     16
4         17   1082
31        19      9
5         21    476
33        23      4
6         25    270
40        27      1
7         29    213
39        31      2
8         33    133
9         37    113
10        41    101
46        43      1
12        45     95
44        47      1
13        49     74
43        51      1
17        53     59
14        57     69
45        59      1
16        61     64
18        65     58
37        67      2
19        69     51
20        73     27
21        77     23
24        81     19
23        85     19
42        87      1
22        89     20
47        91      1
27        93     16
25        97     18
29       101     14
41       103      1
26       105     17
38       107      2
34       109      3
32       113      8
36       117      2
35       1